In [1]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

def setup_environment():
    """Setup your API KEYs"""
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key=getpass.getpass("Enter your OPENAI API KEY")
        os.environ["OPENAI_API_KEY"]=openai_key

    if not os.environ.get("LANGSMITH_API_KEY"):
        langsmith_key=getpass.getpass("Enter your langsmith api key")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"]=langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "Langgraph tutorial"
        else:
            print("skipping langsmith setup")

    print("Environment Setup is Complete")        

In [2]:
setup_environment()

Environment Setup is Complete


In [3]:
# Now Step -1 - Defining State
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages              #This is a method             
from langchain_core.messages import HumanMessage, AIMessage   # these are classes. one with camel casing name are classes

class State(TypedDict):
    # messages will store our conversation history
    # add_messages is a special function that append new message instead of replacing them

    messages: Annotated[list, add_messages]

In [4]:
#getting the LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0.7
)

In [5]:
#Step-3  - lets create a node (which is just a Like a function)
def chatbot_node(state: State)->Dict[str, Any]:
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

In [6]:
# step - 4 - Setup Edges in Graph

#Now for this we will invoke the class "StateGraph with an object"
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot_node)
#lets add edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

simple_chatbot = graph_builder.compile()

In [7]:
#Now tpo genrate the message for graph, we can do below
initial_state = {
    "messages": [HumanMessage(content="Hello, how are you")]
}
result = simple_chatbot.invoke(initial_state)

In [8]:
#now for graph, we should always put the message in the loop for iteration
for i, message in enumerate(result['messages']):
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")

    if isinstance(message, AIMessage):
        print(f"AI: {message.content}")

Human: Hello, how are you
AI: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [9]:
#Now the above part let us just store it in a function and just re-use it
def test_simple(user_input):
    initial_state = {
    "messages": [HumanMessage(content="Hello, how are you")]
    }
    result = simple_chatbot.invoke(initial_state)

    for i, message in enumerate(result['messages']):
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")

        if isinstance(message, AIMessage):
            print(f"AI: {message.content}")

In [10]:
#Now lets us again add memory to the chatbot
#lets use sql_lite database
!pip install langgraph-checkpoint-sqlite



In [11]:
from langgraph.checkpoint.memory import MemorySaver
checkpointer=MemorySaver()

In [12]:
graph_builder_with_memory = StateGraph(State)

graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=checkpointer)

In [13]:
def test_simple(user_input):
    config={"configurable":{"thread_id":"conversation_1"}}   #Threads per conversation
    initial_state={

        "messages":[HumanMessage(content=user_input)]
    }
    result=chatbot_with_memory.invoke(initial_state,config)
    for i, message in enumerate(result['messages']):
        if isinstance(message,HumanMessage):
            print(f"Human:{message.content}")

        if isinstance(message,AIMessage):
            print(f"AI:{message.content}")
    return result    

In [14]:
user_prompt=input("Enter your prompt:")
test_simple(user_prompt)

Human:which is the faster bike in the world
AI:As of the latest information available, the Dodge Tomahawk is often cited as one of the fastest motorcycles in the world. It features a V10 engine from a Dodge Viper and is claimed to have a top speed of around 350 mph (560 km/h). However, it should be noted that the Tomahawk is more of a concept vehicle and not street-legal.

For production motorcycles, the Kawasaki Ninja H2R is frequently mentioned as the fastest. It's a track-only bike with a supercharged 998cc inline-four engine, capable of reaching speeds over 240 mph (386 km/h).

Keep in mind that these figures are subject to change as new models are developed and performance enhancements are made in the motorcycle industry.


{'messages': [HumanMessage(content='which is the faster bike in the world', additional_kwargs={}, response_metadata={}, id='33f9ea67-a547-471a-8154-84f003bba61f'),
  AIMessage(content="As of the latest information available, the Dodge Tomahawk is often cited as one of the fastest motorcycles in the world. It features a V10 engine from a Dodge Viper and is claimed to have a top speed of around 350 mph (560 km/h). However, it should be noted that the Tomahawk is more of a concept vehicle and not street-legal.\n\nFor production motorcycles, the Kawasaki Ninja H2R is frequently mentioned as the fastest. It's a track-only bike with a supercharged 998cc inline-four engine, capable of reaching speeds over 240 mph (386 km/h).\n\nKeep in mind that these figures are subject to change as new models are developed and performance enhancements are made in the motorcycle industry.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 15, 

In [15]:
# Now lets make it as an API
#Tool integration
#so that the agent can interact with outside world.

from langchain_core.tools import tool
import math

@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression safely.
    
    Args:
        expression: A mathematical expression to evaluate (e.g., "2 + 3 * 4")
        
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation of mathematical expressions
        # Only allow basic math operations
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        allowed_names.update({"abs": abs, "round": round})
        
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

@tool  
def get_current_time() -> str:
    """Get the current time."""
    from datetime import datetime
    return f"Current time is: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Test our tools
print("🧮 Testing Calculator Tool:")
print(calculator.invoke({"expression": "2 + 3 * 4"}))
print(calculator.invoke({"expression": "sqrt(16) + 5"}))

print("\n⏰ Testing Time Tool:")
print(get_current_time.invoke({}))


🧮 Testing Calculator Tool:
The result of 2 + 3 * 4 is 14
The result of sqrt(16) + 5 is 9.0

⏰ Testing Time Tool:
Current time is: 2025-06-29 15:03:43


#Create a Tool-Enabled Chatbot

Now we need to:
1. Bind tools to our LLM
2. Create a tool-calling node
3. Add conditional logic to decide when to use tools

In [16]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import ToolMessage

# Step 1: Create LLM with tools
tools = [calculator, get_current_time]
llm_with_tools = llm.bind_tools(tools)

# Step 2: Create nodes
def chatbot_with_tools(state: State) -> Dict[str, Any]:
    """Chatbot that can use tools"""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Step 3: Create tool node using LangGraph's prebuilt ToolNode
tool_node = ToolNode(tools)

print("✅ Tool-enabled chatbot components created!")
print("- LLM knows about our tools")
print("- Chatbot node can generate tool calls") 
print("- Tool node can execute the tools")


✅ Tool-enabled chatbot components created!
- LLM knows about our tools
- Chatbot node can generate tool calls
- Tool node can execute the tools


Add Conditional Logic

We need to route between:
- **Tools**: If the LLM wants to use a tool
- **End**: If the LLM gives a final response

In [17]:
from typing import Literal

def should_continue(state: State) -> Literal["tools", "__end__"]:
    """
    Determine if we should use tools or end the conversation.
    
    Returns:
        "tools" if the last message has tool calls
        "__end__" if we should end
    """
    messages = state["messages"]
    last_message = messages[-1]                    #we are using -1 so that we can fetch the last message(previous) from the user. This is how it works.
    
    # If the last message has tool calls, we should use tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:               #we are using hasattr which is Hash String as we had used dictionary type before
        return "tools"
    else:
        return "__end__"

print("✅ Conditional logic created!")
print("This function decides whether to:")
print("- Use tools (if LLM made tool calls)")
print("- End conversation (if LLM gave final response)")

✅ Conditional logic created!
This function decides whether to:
- Use tools (if LLM made tool calls)
- End conversation (if LLM gave final response)


Build the Tool-Enabled Graph

Now let's put it all together:

In [18]:
#creating the graph 
tools_graph_builder = StateGraph(State)

#adding nodes
tools_graph_builder.add_node("chatbot", chatbot_with_tools)
tools_graph_builder.add_node("tools", tool_node )

#add edges
tools_graph_builder.add_edge(START, "chatbot")

#add conditional edges
tools_graph_builder.add_conditional_edges(
    "chatbot", #from the chatbot node
    should_continue,  # use this function to decide what logic to use or end
    {
        "tools":"tools", # If should_continue returns "tools", go to tools node
        "_end":END       # If should_continue returns "__end__", end the graph
    }
)

# after tools, go back to chatbot
tools_graph_builder.add_edge("tools",END)

#Compile with memory
tool_chatbot = tools_graph_builder.compile(checkpointer=checkpointer)

print("✅ Tool-enabled chatbot with memory created!")
print("Flow: START → chatbot → [tools OR end] → (if tools) → chatbot → ...")


✅ Tool-enabled chatbot with memory created!
Flow: START → chatbot → [tools OR end] → (if tools) → chatbot → ...


### Test the Tool-Enabled Chatbot

Let's test our enhanced chatbot with some math problems and time queries:

In [19]:
def test_tool_chatbot():
    print("🛠️ Testing Tool-Enabled Chatbot")
    print("=" * 50)
    
    config = {"configurable": {"thread_id": "tool_test_1"}}
    
    # Test 1: Math calculation
    print("📝 Test 1: Math Calculation")
    result1 = tool_chatbot.invoke(
        {"messages": [HumanMessage(content="What's 15 * 7 + sqrt(144)?")]},
        config
    )
    
    for message in result1["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 AI: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool: {message.content}")
    
    print("\n📝 Test 2: Current Time")
    result2 = tool_chatbot.invoke(
        {"messages": [HumanMessage(content="What time is it right now?")]},
        config
    )
    
    # Show only new messages
    new_messages = result2["messages"][len(result1["messages"]):]
    for message in new_messages:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 AI: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool: {message.content}")
    
    print("\n✅ Tool integration working perfectly!")
    return result2

# Test the tool-enabled chatbot
tool_result = test_tool_chatbot()

🛠️ Testing Tool-Enabled Chatbot
📝 Test 1: Math Calculation
👤 Human: What's 15 * 7 + sqrt(144)?
🤖 AI: [Calling tool: calculator]
🔧 Tool: The result of 15 * 7 + sqrt(144) is 117.0

📝 Test 2: Current Time
👤 Human: What time is it right now?
🤖 AI: [Calling tool: get_current_time]
🔧 Tool: Current time is: 2025-06-29 15:03:46

✅ Tool integration working perfectly!


Part 5: Human-in-the-Loop Workflows {#part5}

Sometimes AI agents need human oversight or approval before taking actions. LangGraph makes this easy with **interrupts**.

### What are Interrupts?

Interrupts pause the graph execution at specific nodes, allowing humans to:
- Review what the agent plans to do
- Modify the state if needed
- Approve or reject actions
- Provide additional guidance

### Step 1: Create an Agent that Asks for Help

Let's create an agent that can request human assistance when it's unsure:

In [20]:
# Enhanced State with human assistance flag
class HumanLoopState(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool  # Flag to request human help

In [21]:
@tool
def request_human_help(question: str) -> str:
    """
    Request help from a human supervisor.
    
    Args:
        question: The question or situation where human help is needed
        
    Returns:
        Confirmation that help has been requested
    """
    return f"Human help requested for: {question}"

# Add the new tool to our toolkit
human_tools = [calculator, get_current_time, request_human_help]
# now we will bind
llm_with_human_tools = llm.bind_tools(human_tools)


In [23]:
#create a node with human assistant logic
#Lets create a chatbot/ LLM node
def chatbot_with_human_help(state: HumanLoopState) -> Dict[str, Any]:
    messages - state["messages"]
    response = llm_with_human_tools.invoke(messages)
    ask_human = False
    if hasattr(response, "tool_calls"):
        for tool_call in response.tool_calls:
            if tool_call['name'] == 'request_human_help':
                ask_human = True
    return {"messages": [response], "ask_human": ask_human}

#now lets create a human node
def human_node(state: HumanLoopState) -> Dict[str, Any]:
    """
    Human intervention node - this is where humans can provide input
    """
    # In a real application, this would wait for human input
    # For this demo, we'll simulate human response
    last_message = state["messages"][-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        tool_call = last_message.tool_calls[0]
        if tool_call['name'] == 'request_human_help':
            # Simulate human response
            human_response = "I've reviewed your question. Please proceed with the calculation and provide a detailed explanation."
            tool_message = ToolMessage(
                content = human_response,
                tool_call_id = tool_call['id']
            )
            return{"message": [tool_message],
                   "ask_human": False}
        
    return {"ask_human": False}   


print("✅ Human-in-the-loop nodes created!")
print("- chatbot_with_human_help: Can request human assistance")
print("- human_node: Handles human intervention")

✅ Human-in-the-loop nodes created!
- chatbot_with_human_help: Can request human assistance
- human_node: Handles human intervention


### Step 3: Build the Human-in-the-Loop Graph
### Now we will create edges and conditional edges accordinly 

In [34]:
def route_human_loop(state: HumanLoopState) -> Literal["human", "tools", "__end__"]:
    """Route based on whether human help is needed or tools should be called"""
    
    if state.get("ask_human", False):
        return "human"
    
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check for tool calls (excluding request_human_help)
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call['name'] != 'request_human_help':
                return "tools"
            else:
                return "__end__"

# Build the graph
human_loop_builder = StateGraph(HumanLoopState)

# Add nodes
human_loop_builder.add_node("chatbot", chatbot_with_human_help)
human_loop_builder.add_node("tools", ToolNode(human_tools))
human_loop_builder.add_node("human", human_node)

# Add edges
human_loop_builder.add_edge(START, "chatbot")

# Add conditional routing/ edges
human_loop_builder.add_conditional_edges(
    "chatbot",
    route_human_loop,
    {
        "human": "human",
        "tools": "tools", 
        "__end__": END
    }
)

# After human or tools, go back to END (because if we got back to Chatbot, it will be going to infinite loop)
human_loop_builder.add_edge("human", END)
human_loop_builder.add_edge("tools", END)

# Compile with interrupt before human node
human_loop_chatbot = human_loop_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["human"]  # This pauses execution before human node
)

print("✅ Human-in-the-loop chatbot created!")
print("Key feature: interrupt_before=['human'] pauses for human input")


✅ Human-in-the-loop chatbot created!
Key feature: interrupt_before=['human'] pauses for human input


### Step 4: Test Human-in-the-Loop

Let's test the interrupt functionality:


In [ ]:
def test_human_loop():
    print("🤝 Testing Human-in-the-Loop")
    print("=" * 40)
    
    config = {"configurable": {"thread_id": "human_loop_test"}}
    
    # Request that requires human help
    print("📝 Asking agent to request human help:")
    
    initial_result = human_loop_chatbot.invoke(
        {"messages": [HumanMessage(content="I need help with a complex decision. Can you request human assistance?")]},
        config
    )
    
    for message in initial_result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Requesting human help]")
            else:
                print(f"🤖 AI: {message.content}")
    
    # Check if execution was interrupted
    state = human_loop_chatbot.get_state(config)
    print(f"\n🔍 Current state:")
    print(f"Next node to execute: {state.next}")
    print(f"Ask human flag: {state.values.get('ask_human', False)}")
    
    if state.next == ("human",):
        print("\n✅ Execution interrupted! Human intervention required.")
        print("In a real app, a human would now review and provide input.")
        
        # Continue execution (simulating human approval)
        print("\n▶️ Continuing execution (simulating human input)...")
        final_result = human_loop_chatbot.invoke(None, config)
        
        # Show the final messages
        new_messages = final_result["messages"][len(initial_result["messages"]):]
        for message in new_messages:
            if isinstance(message, ToolMessage):
                print(f"👨‍💼 Human: {message.content}")
            elif isinstance(message, AIMessage):
                print(f"🤖 AI: {message.content}")
    
    return final_result

# Test the human-in-the-loop functionality
human_loop_result = test_human_loop()